import pandas as pd
import os

# Check if running in Colab
IN_COLAB = 'COLAB_GPU' in os.environ

if IN_COLAB:
    # For Google Colab
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Clone the repository to get the data file
    !git clone https://github.com/tkim/unsloth-fine-tuning.git /content/unsloth-fine-tuning
    
    # Set the correct path for Colab
    csv_path = "/content/unsloth-fine-tuning/data/training-data-1.csv"
else:
    # For local environment
    csv_path = "data/training-data-1.csv"

# Check if file exists
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} rows from {csv_path}")
    print(f"Columns: {df.columns.tolist()}")
    print("\nFirst few rows:")
    print(df.head())
else:
    print(f"File not found at {csv_path}")
    print("Creating sample data...")
    # Create sample data if file doesn't exist
    import numpy as np
    sample_data = []
    for i in range(100):
        history = [np.random.randint(1, 100) for _ in range(5)]
        next_nums = [np.random.randint(1, 100) for _ in range(5)]
        sample_data.append({
            'input': ', '.join(map(str, history)),
            'output': ', '.join(map(str, next_nums))
        })
    df = pd.DataFrame(sample_data)
    print(f"Created sample data with {len(df)} rows")
    print(df.head())

## ⚡ Quick Start Instructions

To run this notebook successfully:

1. **Runtime Setup**: Go to `Runtime` → `Change runtime type` → Select `GPU` (T4 or better)
2. **Run All**: Click `Runtime` → `Run all` to execute all cells in order
3. **Manual Execution**: If running cells individually, run them in order from top to bottom

**Important**: The cells must be run in sequence. Each cell depends on variables created in previous cells.

### 🔧 Troubleshooting Common Issues:

- **"name 'df' is not defined"**: Run the data loading cell first
- **"You cannot perform fine-tuning on purely quantized models"**: Run the LoRA adapter cell before training
- **Out of memory**: Reduce batch size or use a smaller model
- **File not found**: The notebook will automatically create sample data if the CSV file is missing

In [ ]:
import pandas as pd
import os
import numpy as np

# Check if running in Colab
IN_COLAB = 'COLAB_GPU' in os.environ

if IN_COLAB:
    # For Google Colab
    from google.colab import drive
    drive.mount('/content/drive')

    # Clone the repository to get the data file (if not already cloned)
    if not os.path.exists('/content/unsloth-fine-tuning'):
        !git clone https://github.com/tkim/unsloth-fine-tuning.git /content/unsloth-fine-tuning

    # Set the correct path for Colab
    csv_path = "/content/unsloth-fine-tuning/data/training-data-1.csv"
else:
    # For local environment
    csv_path = "data/training-data-1.csv"

# Check if file exists
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"✓ Loaded {len(df)} rows from {csv_path}")
    print(f"✓ Columns: {df.columns.tolist()}")
    print("\nFirst few rows:")
    print(df.head())
else:
    print(f"⚠️ File not found at {csv_path}")
    print("Creating sample data for demonstration...")
    # Create sample data if file doesn't exist
    sample_data = []
    for i in range(100):
        history = [np.random.randint(1, 100) for _ in range(5)]
        next_nums = [np.random.randint(1, 100) for _ in range(5)]
        sample_data.append({
            'input': ', '.join(map(str, history)),
            'output': ', '.join(map(str, next_nums))
        })
    df = pd.DataFrame(sample_data)
    print(f"✓ Created sample data with {len(df)} rows")
    print("\nSample data:")
    print(df.head())

# Verify data is ready
print(f"\n✓ Data ready for training: {len(df)} examples")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Loaded 1847 rows from /content/unsloth-fine-tuning/data/training-data-1.csv
✓ Columns: ['number_1', 'number_2', 'number_3', 'number_4', 'number_5', 'mb']

First few rows:
   number_1  number_2  number_3  number_4  number_5  mb
0        20        36        37        48        67  16
1        14        39        43        44        67  19
2         9        38        47        49        68  25
3        15        16        18        39        59  17
4         5        11        25        27        64  13

✓ Data ready for training: 1847 examples


In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.7.7: Fast Mistral patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from datasets import Dataset

# Check if df is defined
if 'df' not in globals():
    print("Error: DataFrame 'df' is not defined. Please run the data loading cell first.")
    print("Creating sample data...")
    import pandas as pd
    import numpy as np
    sample_data = []
    for i in range(100):
        history = [np.random.randint(1, 100) for _ in range(5)]
        next_nums = [np.random.randint(1, 100) for _ in range(5)]
        sample_data.append({
            'input': ', '.join(map(str, history)),
            'output': ', '.join(map(str, next_nums))
        })
    df = pd.DataFrame(sample_data)
    print(f"Created sample data with {len(df)} rows")

def format_prompt(row):
    """
    Format CSV row data for number combination prediction.
    Adjust column names based on your CSV structure.
    """
    # Example format - adjust based on your CSV columns
    # Assuming columns like 'history', 'pattern', 'next_numbers'
    if 'input' in row and 'output' in row:
        return f"### Input: {row['input']}\n### Output: {row['output']}<|endoftext|>"
    else:
        # Adjust this based on your actual CSV columns
        # For number prediction, you might have historical numbers as input
        # and the next numbers as output
        input_text = str(row.get('history', row.get('previous_numbers', '')))
        output_text = str(row.get('next_numbers', row.get('prediction', '')))
        return f"### Input: {input_text}\n### Output: {output_text}<|endoftext|>"

# Convert DataFrame to formatted dataset
formatted_data = [format_prompt(row) for _, row in df.iterrows()]
dataset = Dataset.from_dict({"text": formatted_data})

print(f"Created dataset with {len(dataset)} examples")
print(f"Sample formatted prompt:\n{formatted_data[0]}")

Created dataset with 1847 examples
Sample formatted prompt:
### Input: 
### Output: <|endoftext|>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Check if model has LoRA adapters
if 'model' not in globals():
    raise ValueError("Model not loaded. Please run the model loading cell first.")

# Check if model has peft config (LoRA adapters)
if not hasattr(model, 'peft_config'):
    print("⚠️ Model doesn't have LoRA adapters. Adding them now...")
    from unsloth import FastLanguageModel
    model = FastLanguageModel.get_peft_model(
        model,
        r=64,  # LoRA rank - higher = more capacity, more memory
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
        lora_dropout=0,  # Supports any, but = 0 is optimized
        bias="none",     # Supports any, but = "none" is optimized
        use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
        random_state=3407,
        use_rslora=False,  # Rank stabilized LoRA
        loftq_config=None, # LoftQ
    )
    print("✓ LoRA adapters added successfully")

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

print("✓ Trainer initialized successfully")

⚠️ Model doesn't have LoRA adapters. Adding them now...
✓ LoRA adapters added successfully


Unsloth: Tokenizing ["text"]:   0%|          | 0/1847 [00:00<?, ? examples/s]

✓ Trainer initialized successfully


In [ ]:
# Test the fine-tuned model for number combination prediction
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt for number prediction
# Adjust this based on your CSV data format
messages = [
    {"role": "user", "content": "Predict the next number combination based on: 12, 34, 56, 78, 90"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda" if torch.cuda.is_available() else "cpu")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    use_cache=True,
    temperature=0.3,  # Lower temperature for more deterministic predictions
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print("Model prediction:")
print(response)

# Extract just the prediction part
if "### Output:" in response:
    prediction = response.split("### Output:")[-1].split("<|endoftext|>")[0].strip()
    print(f"\nExtracted prediction: {prediction}")

Model prediction:
<|user|> Predict the next number combination based on: 12, 34, 56, 78, 90<|end|><|assistant|>,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,',name_id_1',name_id_2',name_id_3',name_id_4',name_id_5',name_id_6',name_id_7',name_id_8',name_id_9',name_id_10',name_id_11,name_id_12',name_id_13


In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)